In [1]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import numpy as np
import math

In [35]:
import data_file
male_list = data_file.male_list
female_list = data_file.female_list
male_professions = data_file.male_professions
female_professions = data_file.female_professions
df_professions = data_file.df_professions

In [42]:
male_list

[['laborer',
  'courier',
  'repairer',
  'cleaner',
  'conductor',
  'programmer',
  'chef',
  'helper',
  'estimator',
  'porter'],
 ['pipelayer',
  'mason',
  'conductor',
  'installer',
  'courier',
  'cleaner',
  'announcer',
  'electrician',
  'helper',
  'programmer'],
 ['cabinetmaker',
  'carpenter',
  'porter',
  'chef',
  'engineer',
  'conductor',
  'machinist',
  'brickmason',
  'pilot',
  'pipelayer'],
 ['painter',
  'courier',
  'drafter',
  'cabinetmaker',
  'programmer',
  'architect',
  'laborer',
  'dishwasher',
  'cleaner',
  'mason'],
 ['painter',
  'dishwasher',
  'announcer',
  'architect',
  'engineer',
  'repairer',
  'brickmason',
  'machinist',
  'chef',
  'courier'],
 ['mason',
  'drafter',
  'clergy',
  'electrician',
  'laborer',
  'cleaner',
  'roofer',
  'carpenter',
  'porter',
  'engineer'],
 ['brickmason',
  'laborer',
  'pipelayer',
  'installer',
  'painter',
  'carpenter',
  'engineer',
  'estimator',
  'clergy',
  'dishwasher'],
 ['architect',
  'm

In [30]:
for i in male_professions.label1.unique():
    print(i)

mason
pipelayer
brickmason
helper
mechanic
carpenter
electrician
roofer
millwright
installer
repairer
painter
firefighter
machinist
conductor
cabinetmaker
pilot
laborer
engineer
cleaner
programmer
courier
porter
announcer
estimator
architect
chef
clergy
drafter
dishwasher


In [4]:
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# # import dataset
# jobs_df =pd.DataFrame(["technician", "accountant", "supervisor", "engineer", "worker", "educator", "clerk", "counselor", "inspector", "mechanic", "manager", "therapist", "administrator", "salesperson", "receptionist", "librarian", "advisor", "pharmacist", "janitor", "psychologist", "physician", "carpenter", "nurse", "investigator", "bartender", "specialist", "electrician", "officer", "pathologist", "teacher", "lawyer", "planner", "practitioner", "plumber", "instructor", "surgeon", "veterinarian", "paramedic", "examiner", "chemist", "machinist", "appraiser", "nutritionist", "architect", "hairdresser", "baker", "programmer", "paralegal", "hygienist", "scientist"])
# job_list = jobs_df[0]
# job_list

In [43]:
# # gendered_word_list
# gendered_word_list = [["she", "he"],["female", "male"], ["woman", "man"], ["girl", "boy"], ["sister", "brother"], ["daughter", "son"], ["mother", "father"]]
XX = ["female", "woman", "girl", "sister", "daughter", "mother", "aunt", "grandmother", "daughter"]
YY = ["male", "man", "boy", "brother", "son", "father", "uncle", "grandfather", "son"]

In [45]:
YY

['male',
 'man',
 'boy',
 'brother',
 'son',
 'father',
 'uncle',
 'grandfather',
 'son']

In [7]:
templates = ["this is the _.", "that is the _.", "there is the _.", "here is the _.", "the _ is here.", "the _ is there.", "the _ is a person.", "it is the _.","the _ is a [MASK].", "the _ is an engineer.", "the _ is a nurse with superior technical skills."]

In [8]:
temps = ["temp1", "temp2", "temp3", "temp4","temp5", "temp6", "temp7", "temp8", "temp9", "temp10", "temp11"]

In [9]:
len(templates)

11

In [10]:
def get_index(sentence, word):
    toks = tokenizer(sentence).input_ids
    wordpieces = tokenizer(word).input_ids
#     print(toks)
    word = wordpieces[1] # use first wordpiece
    for i, t in enumerate(toks):
        if t == word:
            return i

In [11]:
# SWP first embedding
def sentence_embedding(template, word):
    sentence = template.replace("_", word)
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    token_embeddings = last_hidden_states
    input_mask_expanded = inputs.attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    start = get_index(sentence, word)
    embeddings = token_embeddings[0][start]
    return embeddings.cpu().detach().numpy()



In [12]:
def cossim(x, y):
    return np.dot(x, y) / math.sqrt(np.dot(x, x) * np.dot(y, y))

In [13]:
def construct_cossim_lookup(XY, AB):
    """
    XY: mapping from target string to target vector (either in X or Y)
    AB: mapping from attribute string to attribute vectore (either in A or B)
    Returns an array of size (len(XY), len(AB)) containing cosine similarities
    between items in XY and items in AB.
    """

    cossims = np.zeros((len(XY), len(AB)))
    for xy in XY:
        for ab in AB:
            cossims[xy, ab] = cossim(XY[xy], AB[ab])
    return cossims


In [14]:
def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [15]:

def s_XAB(A, s_wAB_memo):
    return s_wAB_memo[A].sum()

In [16]:
def s_wAB(X, Y, cossims):
    """
    Return vector of s(w, A, B) across w, where
        s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
    """
#     print((cossims[X, :].mean(axis=0) - cossims[Y, :].mean(axis=0)).shape)
    return cossims[X, :].mean(axis=0) - cossims[Y, :].mean(axis=0)

In [17]:
# def s_wAB_df(X, Y, cossims):
#     """
#     Return vector of s(w, A, B) across w, where
#         s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
#     """
# #     print(cossims[:, A].mean(axis=1))
#     df1 = pd.DataFrame(cossims[X, :].mean(axis=0))
#     df2 = pd.DataFrame(cossims[Y, :].mean(axis=0))
#     return df1, df2


def s_XAB_df(A, B, s_wAB_memo):
    df1 = pd.DataFrame(s_wAB_memo[A])
    df2 = pd.DataFrame(s_wAB_memo[B])
    return df1, df2

In [18]:
def s_XYAB(A, B, s_wAB_memo):
    r"""
    Given indices of target concept X and precomputed s_wAB values,
    the WEAT test statistic for p-value computation.
    """
    return s_XAB(A, s_wAB_memo) - s_XAB(B, s_wAB_memo)

In [19]:
# def WEAT_test(X, Y, A, B, n_samples, cossims, parametric=False):
def WEAT_test(X, Y, A, B, n_samples, cossims):
    ''' Compute the p-val for the permutation test, which is defined as
        the probability that a random even partition X_i, Y_i of X u Y
        satisfies P[s(X_i, Y_i, A, B) > s(X, Y, A, B)]
    '''
    X = np.array(list(X), dtype=np.int)
    Y = np.array(list(Y), dtype=np.int)
    A = np.array(list(A), dtype=np.int)
    B = np.array(list(B), dtype=np.int)

    assert len(X) == len(Y)
    size = len(X)
    s_wAB_memo = s_wAB(X, Y, cossims=cossims)
#     print(s_wAB_memo)
    XY = np.concatenate((X, Y))

#     if parametric:
#     log.info('Using parametric test')
    s = s_XYAB(A, B, s_wAB_memo)
    return s


def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [20]:
def get_effect_size(df1, df2, k=0):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_

In [21]:
template_score_dict = {}
for ind, template in enumerate(templates):
    score_dict = {}
    attribute_template = template
    target_template = template
    X = {"x" + str(j): sentence_embedding(attribute_template, j) for j in XX}
    Y = {"y" + str(j): sentence_embedding(attribute_template, j) for j in YY}
    (X, Y) = convert_keys_to_ints(X, Y)
    XY = X.copy()
    XY.update(Y)
    X = np.array(list(X), dtype=np.int)
    Y = np.array(list(Y), dtype=np.int)
    for i in range(len(female_list)):
        AA = female_list[i]
    #     print(AA)
    #     print(XX)
        BB = male_list[i]

        A = {"a" + str(j): sentence_embedding(target_template, j) for j in AA}
        B = {"b" + str(j): sentence_embedding(target_template, j) for j in BB}

        (A, B) = convert_keys_to_ints(A, B)


        AB = A.copy()
        AB.update(B)

        cossims = construct_cossim_lookup(XY, AB)
        A = np.array(list(A), dtype=np.int)
        B = np.array(list(B), dtype=np.int)


        s_wAB_memo = s_wAB(X, Y, cossims=cossims)
        df1,df2 = s_XAB_df(A, B, s_wAB_memo)
        effect_size = get_effect_size(df1, df2)
        score_dict[i] = effect_size
        print(score_dict[i])
    template_score_dict[temps[ind]] = score_dict
# log.info("pval: %g", pval)

# log.info("computing effect size...")
# esize = effect_size(X, Y, A, B, cossims=cossims)
# log.info("esize: %g", esize)

0.5990485031897409
0.6714068637301815
1.1631737950151444
0.4300088064734563
1.0807484633410291
0.728444433582044
0.5967151799449145
0.9699216422992344
0.6764622523800463
1.0017584512258777
0.9014876829399008
0.9651974526786529
0.9453429085731987
0.8318776437875482
0.6902093367385614
0.5537827970200967
0.7582517880853172
0.6479462511409895
0.5757022345892697
0.3803179553070429
0.5165489614260299
0.6141997131591962
1.2558204830200008
0.3195518057342596
1.085944813136749
0.6351491168006498
0.48156724495854947
0.9409978670467799
0.5937688704144524
1.0384432796198535
0.8576356289772351
0.9009797381938763
0.8539576566630959
0.8045660131944413
0.619059379006087
0.5190840718699152
0.6637209690199731
0.6769517532374608
0.5767742268934358
0.4241470866271133
0.5425800457422781
0.5389729441356057
1.258937457922151
0.020865742453891768
0.9988570922077628
0.24451862086510393
0.44158565057387833
0.70134961837719
0.18831162934032222
1.2223636391188837
0.6044390822436191
0.9361498226970248
0.7180995861

In [22]:
template_score_dict

{'temp1': {0: 0.5990485031897409,
  1: 0.6714068637301815,
  2: 1.1631737950151444,
  3: 0.4300088064734563,
  4: 1.0807484633410291,
  5: 0.728444433582044,
  6: 0.5967151799449145,
  7: 0.9699216422992344,
  8: 0.6764622523800463,
  9: 1.0017584512258777,
  10: 0.9014876829399008,
  11: 0.9651974526786529,
  12: 0.9453429085731987,
  13: 0.8318776437875482,
  14: 0.6902093367385614,
  15: 0.5537827970200967,
  16: 0.7582517880853172,
  17: 0.6479462511409895,
  18: 0.5757022345892697,
  19: 0.3803179553070429},
 'temp2': {0: 0.5165489614260299,
  1: 0.6141997131591962,
  2: 1.2558204830200008,
  3: 0.3195518057342596,
  4: 1.085944813136749,
  5: 0.6351491168006498,
  6: 0.48156724495854947,
  7: 0.9409978670467799,
  8: 0.5937688704144524,
  9: 1.0384432796198535,
  10: 0.8576356289772351,
  11: 0.9009797381938763,
  12: 0.8539576566630959,
  13: 0.8045660131944413,
  14: 0.619059379006087,
  15: 0.5190840718699152,
  16: 0.6637209690199731,
  17: 0.6769517532374608,
  18: 0.5767742

In [23]:
import json
a_file = open("seat_SWP_templates.json", "w")
json.dump(template_score_dict, a_file)
a_file.close()